<a href="https://colab.research.google.com/github/humzkhan/Adaptive_learning/blob/main/Thesis_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from skopt import gp_minimize
from skopt.plots import plot_convergence

# Example black-box function (toy): we want to minimize this function
def black_box_function(x):
    """ A simple test function: f(x) = x^2 * sin(x)
        x is a list or array, but we use x[0] because it's 1D.
    """
    val = (x[0]**2) * np.sin(x[0])
    return val


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # Approve the authentication prompts as needed

Mounted at /content/drive


In [4]:

# Step 2: Define the base path to your folder
import os

base_path = '/content/drive/MyDrive/Thesis_Updated/singleLayerData'
# If your Drive is mounted as "My Drive" (with a space), use:
# base_path = '/content/drive/My Drive/Thesis/Data Files/Single layer samples'

# Step 3: Recursively find all files (CSV and Excel) in the base folder
import glob
import pandas as pd

# Dictionary to store DataFrames
all_dfs = {}

# Find all CSV and Excel files
csv_files = glob.glob(os.path.join(base_path, '**', '*.csv'), recursive=True)
xls_files = glob.glob(os.path.join(base_path, '**', '*.xls'), recursive=True)
xlsx_files = glob.glob(os.path.join(base_path, '**', '*.xlsx'), recursive=True)
excel_files = xls_files + xlsx_files

print(f"Found {len(csv_files)} CSV files and {len(excel_files)} Excel files.")

# Step 4: Load all files into pandas DataFrames
# Load CSV files (skip first 14 lines of the header)
for file_path in csv_files:
    try:
        relative_path = os.path.relpath(file_path, base_path)
        print(f"Loading CSV: {relative_path} (Skipping first 14 lines)")
        df = pd.read_csv(file_path, skiprows=14)  # Skip first 14 rows
        all_dfs[relative_path] = df
    except Exception as e:
        print(f"Error reading CSV {file_path}: {e}")

# Load Excel files
for file_path in excel_files:
    try:
        relative_path = os.path.relpath(file_path, base_path)
        print(f"Loading Excel: {relative_path}")
        df = pd.read_excel(file_path)
        all_dfs[relative_path] = df
    except Exception as e:
        print(f"Error reading Excel {file_path}: {e}")

# Step 5: Verify loaded DataFrames
print("\nLoaded the following files (relative paths):")
for rel_path in all_dfs:
    print(f"  -> {rel_path}")
    print(all_dfs[rel_path].head())  # Show the first few rows for each DataFrame


Found 37 CSV files and 0 Excel files.
Loading CSV: 4%/PBT/4%-PBT-2.0mm.csv (Skipping first 14 lines)
Loading CSV: 5%/PBT/5%-PBT-2.8mm-B.csv (Skipping first 14 lines)
Loading CSV: 5%/PBT/5%-PBT-2.0mm.csv (Skipping first 14 lines)
Loading CSV: 5%/PBT/5%-PBT-2.8mm-A.csv (Skipping first 14 lines)
Loading CSV: 5%/ABS/5%-ABS-2.8mm-C.csv (Skipping first 14 lines)
Loading CSV: 5%/ABS/5%-ABS-2.8mm-B.csv (Skipping first 14 lines)
Loading CSV: 5%/ABS/5%-ABS-2.8mm-A.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-1.1mm-B-ambient.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-2.8mm-C.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-1.1mm-A-ambient.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-1.1mm-A-compressed.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-1.1mm-B-compressed.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-2.8mm-B.csv (Skipping first 14 lines)
Loading CSV: 3%/ABS/3%-ABS-1.1mm-B-quenched.csv (Skipping first 14 lines)
Loading C

In [5]:
def check_headers(dataframes, required_headers):
    """
    Check each DataFrame for the presence of required headers.

    Parameters:
    - dataframes (dict): Dictionary where keys are file paths and values are DataFrames.
    - required_headers (list): List of required headers to check for.

    Returns:
    - dict: Dictionary with filenames as keys and missing headers as values.
    """
    missing_headers = {}

    for file_path, df in dataframes.items():
        # Normalize the column names: lowercase and remove spaces
        normalized_columns = [col.lower().replace(" ", "") for col in df.columns]

        # Find missing headers
        file_missing = [header.lower().replace(" ", "") for header in required_headers if header.lower().replace(" ", "") not in normalized_columns]

        if file_missing:
            missing_headers[file_path] = file_missing

    return missing_headers

# Define required headers
required_headers = ['A', 'R', 'T', 'SEr', 'SEa', 'SEt']

# Check headers for all DataFrames
missing_headers_info = check_headers(all_dfs, required_headers)

# Display results
if missing_headers_info:
    print("\nFiles missing required headers:")
    for file, missing in missing_headers_info.items():
        print(f"File: {file}")
        print(f"Missing headers: {missing}")
else:
    print("\nAll files have the required headers.")



Files missing required headers:
File: 5%/ABS/5%-ABS-2.8mm-C.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 5%/ABS/5%-ABS-2.8mm-B.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 5%/ABS/5%-ABS-2.8mm-A.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-1.1mm-B-ambient.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-2.8mm-C.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-1.1mm-A-ambient.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-1.1mm-A-compressed.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-1.1mm-B-compressed.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-2.8mm-B.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-1.1mm-B-quenched.csv
Missing headers: ['a', 'r', 't', 'ser', 'sea', 'set']
File: 3%/ABS/3%-ABS-2.8mm-A.csv
Missing headers: ['a', 'r', 

In [6]:
import numpy as np

def add_missing_headers_flexible(dataframes):
    """
    Add missing headers to DataFrames by searching flexibly for S11 and S21 in column headers.

    Parameters:
    - dataframes (dict): Dictionary where keys are file paths and values are DataFrames.

    Returns:
    - dict: Updated dictionary of DataFrames with missing headers added.
    """
    updated_files = []  # To track files that were updated

    for file_path, df in dataframes.items():
        # Normalize column names for consistent matching
        df.columns = [col.lower().replace(" ", "").replace("(linm)", "") for col in df.columns]

        # Flag to track if any update was made
        updated = False

        # Attempt to locate 'S11' and 'S21' columns
        s11_col = next((col for col in df.columns if "s11" in col), None)
        s21_col = next((col for col in df.columns if "s21" in col), None)

        # Check and calculate each required column
        if 'r' not in df.columns:
            if s11_col:
                df['r'] = df[s11_col] ** 2
                updated = True
            else:
                print(f"Missing 'S11' column for calculating 'R' in {file_path}")
                continue

        if 't' not in df.columns:
            if s21_col:
                df['t'] = df[s21_col] ** 2
                updated = True
            else:
                print(f"Missing 'S21' column for calculating 'T' in {file_path}")
                continue

        if 'a' not in df.columns:
            if 'r' in df.columns and 't' in df.columns:
                df['a'] = 1 - df['r'] - df['t']
                updated = True
            else:
                print(f"Cannot calculate 'A' in {file_path} due to missing 'R' or 'T'")
                continue

        if 'ser' not in df.columns:
            if 'r' in df.columns:
                df['ser'] = -10 * np.log10(1 - df['r'])
                updated = True
            else:
                print(f"Cannot calculate 'SEr' in {file_path} due to missing 'R'")
                continue

        if 'set' not in df.columns:
            if 't' in df.columns:
                df['set'] = -10 * np.log10(df['t'])
                updated = True
            else:
                print(f"Cannot calculate 'SEt' in {file_path} due to missing 'T'")
                continue

        if 'sea' not in df.columns:
            if 'set' in df.columns and 'ser' in df.columns:
                df['sea'] = df['set'] - df['ser']
                updated = True
            else:
                print(f"Cannot calculate 'SEa' in {file_path} due to missing 'SEt' or 'SEr'")
                continue

        # If updated, track the file
        if updated:
            updated_files.append(file_path)

        # Update DataFrame in the dictionary
        dataframes[file_path] = df

    # Print updated files
    for file_path in updated_files:
        print(f"\nUpdated {file_path}:")
        print(dataframes[file_path].head(1))  # Print only the first row for preview

    return dataframes

# Apply the updated function to all DataFrames
updated_dfs = add_missing_headers_flexible(all_dfs)



Updated 5%/ABS/5%-ABS-2.8mm-C.csv:
   frequency(hz)   tr2s11    tr3s21  tr1data(e')  tr4data(e'')  tr5data(u')  \
0   1.240000e+10  0.87814  0.042179     6.811813      2.641842          1.0   

   tr6data(u'')         r         t        a       ser        set        sea  
0           0.0  0.771131  0.001779  0.22709  6.404125  27.498031  21.093906  

Updated 5%/ABS/5%-ABS-2.8mm-B.csv:
   frequency(hz)    tr2s11    tr3s21  tr1data(e')  tr4data(e'')  tr5data(u')  \
0   1.240000e+10  0.845358  0.052452     6.828651       3.05671          1.0   

   tr6data(u'')        r         t         a       ser        set        sea  
0           0.0  0.71463  0.002751  0.282618  5.445922  25.604749  20.158827  

Updated 5%/ABS/5%-ABS-2.8mm-A.csv:
   frequency(hz)    tr2s11    tr3s21  tr1data(e')  tr4data(e'')  tr5data(u')  \
0   1.240000e+10  0.856208  0.037372     6.844234      2.925287          1.0   

   tr6data(u'')         r         t         a       ser       set        sea  
0           0.0 